# Final ETL Script

This takes the intermediate data and turns it into a final CSV

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import boto3
import sys

# Pip Install PyAthena

Sagemaker Notebooks use a default image that doesn't include PyAthena so you need to use pip to install it.

In [ ]:
!{sys.executable} -m pip install PyAthena

In [ ]:
from pyathena import connect

## Create a PyAthena Connection

In [ ]:
conn = connect(s3_staging_dir='s3://athena-results-c7fhgh8/',
               region_name='us-east-1')

## Pull Songdata from S3 and clean Track ID

In [ ]:
songdata = pd.read_sql("""

select  REVERSE(SUBSTR(REVERSE(SUBSTR(track_id,3)),2)) as track_id
    ,   loudness
    ,   tempo
    ,   artist_familiarity

from \"millionsongdataset-intermediate\".songdata

""", conn)

## Pull Unique Tracks

In [ ]:
unique_tracks = pd.read_sql("""

SELECT *

FROM \"millionsongdataset-intermediate\".unique_tracks

""", conn)

## Pull Spotify Links

In [ ]:
spotify = pd.read_sql("""

SELECT *

FROM \"millionsongdataset-intermediate\".spotify
WHERE spotify_id <> ''

""", conn)

## Merge Songdata and Unique Tracks

In [ ]:
tracks = pd.merge(songdata, unique_tracks, on="track_id")

## Merge the new Merged dataset with Spotify

In [ ]:
data = pd.merge(data, spotify, on="song_id")

## Look at the shape of what is left now after all those manipulations

In [ ]:
data.shape

## Strip out the unwanted columns

In [ ]:
data = data[['track_id','song_title','artist_name','spotify_id','loudness','tempo','artist_familiarity']]

## Use MinMaxScaler to scale the data for visualizations later

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

data[['loudness','tempo','artist_familiarity']] = scaler.fit_transform(data[['loudness','tempo','artist_familiarity']])

data.head(5)

## Drop any rows that have null values

In [ ]:
data = data.dropna(axis=0)

## Query Profanity and Lyrics

In [ ]:
profanity = pd.read_sql("""

WITH joined_set AS (
SELECT
    ly.track_id
    , COUNT(ly.word) AS total_unique_words
    , COALESCE(COUNT(fw.profanity_word),0) AS total_fw
FROM "millionsongdataset-intermediate".lyrics AS ly
LEFT JOIN "millionsongdataset-intermediate".profanity_words AS fw ON ly.word = fw.profanity_word
GROUP BY ly.track_id
)

SELECT
      track_id
    , CASE WHEN total_unique_words > 0 THEN (total_fw/(total_unique_words*1.0))
           ELSE 0.0 
      END AS profanity_pct
    , CASE WHEN total_fw > 0 THEN 1 ELSE 0
    END AS profanity_flag
FROM joined_set
ORDER BY profanity_pct DESC;

""", conn)

## Merge Profanity with Data

In [ ]:
results = pd.merge(data, profanity, on="track_id", how="left")

## Fill NAs with 0

In [ ]:
results[['profanity_pct','profanity_flag']] = results[['profanity_pct','profanity_flag']].fillna(0)

## Save to S3 as CSV in Final Location

In [ ]:
results.to_csv("songdata.csv", index=False)

session = boto3.Session()
s3_client = boto3.client('s3')
s3_client.upload_file("songdata.csv", "millionsongdataset", "songdata/songdata.csv")